In [ ]:
# Graphe de l'ensemble des parkings le 16/01 à 8h et 20h vs 16h

# Parkings

## Corrélation avec les heures de la journée:
Le premiers graphique reprsésente en pourcentage l'occupation des parkings de Montpellierà le lundi 16 janvier à 8h, l'autre à 14h. On constate que le premier graphique affiche des pourcentages plus élevés que le deuxième. On en déduit que les trajets en voitures sont plus souvent utilisés lors des heures de pointe que les heures creuses. Ce rendu s'explique par le fait que chaque salariés doivent aller travailler le matin et rentrer du travail le soir, contrairement à l'après-midi où le trajet en voiture n'est pas solicité.

(graph d'un parking proche  )

## Corrélation avec les jours de la semaine:
O'occupation d'un parking peut varier également en fonction des jours de la semaine. Il est probable que l'occupation soit plus élevée pendant les jours de la semaine ouvrables et plus faible pendant les week-ends.